### Load Dependencies

In [1]:
from utils.HN import *
from utils.model_eval import *

### Define variables
- `class_name2ID` is a dictionary mapping class names to their name. The ID of a species is its position in the Species column of Table 1. It's name is the name given in Table 1. E.g., for the JE-TL19 dataset, : `classID2name = {"Elephant": 0, "Giraffe": 1, "Zebra": 2}`
- The DoR values can be found in Table 12.
- The radii can be found in the Size Avg. column of Table 1. They need to be passed as a dictionary where `key = class ID`, `value = radius`. Again, for the JE-TL19 dataset, the `radii` dict would look as follows: `{0: 62, 1: 81, 2: 49}`


In [2]:
imgs_dir = ""   #TODO: Insert Path to image directory (as downloaded from Zenodo).
model_weights = "" #TODO: Insert Path to the model weights you want to run (as downloaded from Zenodo).
class_name2ID = {"": 0, "": 1, "": 2}    #TODO: Insert class names and IDs as found in Table 1.
dor_thresh = -1.0    #TODO: Define DoR threshold for NMS and computing evaluation metrics (cf. Table 12)
radii = {0: None, 1: None, 2: None}     #TODO: Insert radius for each species in the dataset; cf. Table 1.
output_dir = ""     #TODO: Insert path to directory you want your output stored in.

### Run inference
This cell will run HerdNet on the images from the “imgs_dir” entered above and will automatically create a folder inside it (HerdNet_results) in which the .csv file containing the detections will be saved.
- `dets_file`: This is the path to the .csv file containing the HerdNet predictions. One row represents one detection and is expected to have the following columns:
    - `images`: Contains the file names of the images.
    - `x`: Contains the x-coordinate of the detection.
    - `y`: Contains the y-coordinate of the detection.
    - `scores`: Contains the confidence score of the prediction.
    - `species`: Contains the name of the detected species.

In [3]:
%run -m utils.inference_herdnet {imgs_dir} {model_weights} --size 512 --over 160

from pathlib import Path

dets_file = next(Path(imgs_dir).rglob("*_HerdNet_detections.csv"), None)

### Read HerdNet output and run evaluation pipeline (no input required)

In [ ]:
read_detections_HN(dets_file=dets_file, cls_name2id=class_name2ID, imgs_dir=imgs_dir, dor_thresh=dor_thresh, radii=radii, class_ids=list(radii.keys()), output_dir=output_dir,
                   ann_file=f"{imgs_dir}/test_annotations.json", ann_format="PT_DEFAULT")

class_ID2name = {cid: name for name,cid in class_name2ID.values()}
compute_errors_img_lvl(gt_counts_dir=f"{imgs_dir}/image_counts", pred_counts_dir=f"{output_dir}/detections", class_ids=list(class_ID2name.keys()), 
                           output_dir=output_dir)
compute_em_img_lvl(preds_dir=f"{output_dir}/detections", class_id2name=class_ID2name, task="locate", output_dir=output_dir)    